In [11]:
## This is for plotting reservoir storage
%matplotlib widget
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from matplotlib.widgets import Slider, Button, RadioButtons

reservoirs = [
    'castaic',
    'del_valle',
    'feather',
#     'pyramid',
#     'silverwood',
#     'thermalito',
]

cdec_stations = dict(
    castaic=['CAS'],
    del_valle=['DLV'],
    feather=['ANT', 'DAV', 'FRD', 'ORO'],
    pyramid=['PYM'],
#     silverwood=['SIV'],
    thermalito=['TAB'],
)

metadata = pd.read_csv('../data/reservoirs.csv', index_col='Code')

def get_month_code(month_str):
    """Returns the month integer based off the provided
    three-letter code, i.e. get_month_code('Mar') = 3
    """
    
    if not len(month_str) == 3:
        print('Input must be three-letter code.')
        return
    
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    return months.index(month_str) + 1

def parse_dates(dates):
    """Returns an array of datetime objects for strings in the format Mon-YY."""
    new_arr = []
    for date in dates:
        try:
            month_str, year = date.split('-')
            month = get_month_code(month_str)
            year = int(year)
            # Year is two-digit so make it four
            if year >= 20:
                year = 1900 + year
            else:
                year = 2000 + year

            new_arr.append(datetime.datetime(year, month, 1))
        except:
            new_arr.append(np.nan)
    
    return np.array(new_arr)

def load_monthly_reservoir_data(reservoir, station):
    """Loads the monthly reservoir storage data available
    for the provided reservoir and station.
    """
    if not reservoir in reservoirs:
        print('Reservoir not found')
        return
    
    if not station in cdec_stations[reservoir]:
        print('Station ID not found')
        return
    
    filename = '../data/'+reservoir+'/cdec_'+station+'_monthly.csv'
    
    storage = pd.read_csv(filename, index_col='Date', date_parser=parse_dates,
                          na_values=['--','','m'], parse_dates=True, usecols=[0,1])
    
    storage = storage.dropna(how='any')
    storage = storage / float(metadata.loc[station, 'CAP']) * 100
    
    return pd.to_datetime(storage.index.get_values()), storage.get_values()

default_reservoir='feather'
default_station = 'ORO'

dates, storage = load_monthly_reservoir_data(default_reservoir, default_station)
min_date = dates[0]
max_date = dates[-1]

fig = plt.figure(figsize=(10, 6), dpi=80)
ax = fig.gca()

# Shrink the plot by 25% towards upper-right corner
plt.subplots_adjust(left=0.25, bottom=0.25)
l, = plt.plot(dates, storage, linewidth=2, color='black')

plt.xlabel('Date')
plt.ylabel('Reservoir Storage (% Capacity)')
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
ax.set_title(default_reservoir + ' (' + default_station + ')')

axcolor = 'lightgoldenrodyellow'
axyear_start = plt.axes([0.25, 0.125, 0.65, 0.03], facecolor=axcolor)
axyear_end = plt.axes([0.25, 0.085, 0.65, 0.03], facecolor=axcolor)

syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')

def find_index(dates, year):
    index = 0
    for date in dates:
        if date.year == int(year):
            return index
        index += 1
    return -1

def update(val):
    year_start = syear_start.val
    year_end = syear_end.val
    if not year_start < year_end:
        return
    s_index = find_index(dates, year_start)
    e_index = find_index(dates, year_end + 1)
    l.set_xdata(dates[s_index:e_index])
    l.set_ydata(storage[s_index:e_index])
    ax.relim()
    ax.autoscale_view()
    plt.draw()
syear_start.on_changed(update)
syear_end.on_changed(update)

resetax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(resetax, 'Reset', color=axcolor, hovercolor='0.975')

def reset(event):
    global syear_start, syear_end, station_radio, reservoir_radio, dates, storage
    reservoir = default_reservoir
    station = default_station
    # Load new data
    dates, storage = load_monthly_reservoir_data(reservoir, station)
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(storage)
    ax.set_title(reservoir + ' (' + station + ')')
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    # Reset radio buttons
    rax.clear()
    reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))
    reservoir_radio.on_clicked(set_reservoir)
    stationax.clear()
    station_radio = RadioButtons(stationax, cdec_stations[default_reservoir], active=cdec_stations[default_reservoir].index(default_station))
    station_radio.on_clicked(set_station)
    fig.canvas.flush_events()
button.on_clicked(reset)

rax = plt.axes([0.025, 0.6, 0.115, 0.3], facecolor=axcolor)
reservoir_radio = RadioButtons(rax, reservoirs, active=reservoirs.index(default_reservoir))

stationax = plt.axes([0.025, 0.25, 0.115, 0.15], facecolor=axcolor)
station_radio = RadioButtons(stationax, cdec_stations[default_reservoir], active=cdec_stations[default_reservoir].index(default_station))

def set_station(label):
    global syear_start, syear_end, dates, storage
    reservoir = reservoir_radio.value_selected
    station = label
    # Load new data
    dates, storage = load_monthly_reservoir_data(reservoir, station)
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(storage)
    ax.set_title(reservoir + ' (' + station + ')')
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    fig.canvas.flush_events()
station_radio.on_clicked(set_station)

def set_reservoir(label):
    global syear_start, syear_end, station_radio, dates, storage
    reservoir = label
    station = cdec_stations[reservoir][0]
    # Load new data
    dates, storage = load_monthly_reservoir_data(reservoir, station)
    min_date = dates[0]
    max_date = dates[-1]
    # Plot new data
    l.set_xdata(dates)
    l.set_ydata(storage)
    ax.set_title(reservoir + ' (' + station + ')')
    ax.relim()
    ax.autoscale_view()
    # Reset sliders with new data
    axyear_start.clear()
    syear_start = Slider(axyear_start, 'Start', min_date.year, max_date.year, valinit=min_date.year, valstep=1, valfmt='%1.f')
    syear_start.on_changed(update)
    axyear_end.clear()
    syear_end = Slider(axyear_end, 'End', min_date.year, max_date.year, valinit=max_date.year, valstep=1, valfmt='%1.f')
    syear_end.on_changed(update)
    # Reset site no selector
    stationax.clear()
    station_radio = RadioButtons(stationax, cdec_stations[reservoir], active=0)
    station_radio.on_clicked(set_station)
    fig.canvas.flush_events()
reservoir_radio.on_clicked(set_reservoir)

plt.show()

FigureCanvasNbAgg()

In [12]:
fig = plt.figure(figsize=(12,6), dpi=80)
ax = fig.gca()
ax.set_title('Reservoir Storage')
plt.xlabel('Year')
plt.ylabel('Storage (% of Capacity)')

line_colors = ['black', 'gray', 'green', 'orange', 'blue']

plot_num = 0
for reservoir in reservoirs:
    for station in cdec_stations[reservoir]:
        dates, storage = load_monthly_reservoir_data(reservoir, station)
        line_style = 'solid'
        if plot_num % 3 == 0:
            line_style = 'solid'
        elif plot_num % 2 == 0:
            line_style = 'dotted'
        else:
            line_style = 'dashed'
            
        line_set = int(np.floor(plot_num / 3))
        line_color = line_colors[line_set]
        plt.plot(dates, storage, linestyle=line_style, color=line_color)
        plot_num += 1

FigureCanvasNbAgg()